# load tmodel

In [1]:
import numpy as np
import pyLDAvis
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models import LdaMulticore, LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.mmcorpus import MmCorpus
from gensim.corpora import Dictionary
import pandas as pd
from pathlib import Path
import joblib
import logging
from tqdm.notebook import tqdm

/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/_pytest/mark/structures.py:380: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)
unable to import 'smart_open.gcs', disabling that module
/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict
/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/gensim

In [5]:
logging.basicConfig(level=logging.INFO)
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)

In [3]:
def __num_dist_rows__(array, ndigits=2):
    return array.shape[0] - int((pd.DataFrame(array).sum(axis=1) < 0.99).sum())

In [4]:
corpus = MmCorpus('../models/full_04_01_2020/gensim_corpus.mm')
dct = Dictionary.load('../models/full_04_01_2020/gensim_dct.mm')
lda = LdaModel.load('../models/full_04_01_2020/topic_models/gensim_topic_model250')

tc = lda.get_document_topics(corpus, minimum_probability=0)
tmp = [[v for t, v in r] for r in tqdm(tc)]
tmp_a = np.vstack(tmp)

term_freqs = np.array([dct.cfs[i] for i in range(lda.num_terms)])
np_terms = np.array([lda.id2word[i] for i in range(lda.num_terms)])
doc_lens = [len(i) for i in tqdm(corpus)]

# term_freq = np.array(model.training_data_.sum(axis=0))[0]
vocab = np.array([lda.id2word[i] for i in range(lda.num_terms)])
data = {
    "topic_term_dists": lda.get_topics(),
    "doc_topic_dists": tmp_a,
    "vocab": vocab,
    "term_frequency": term_freqs,
    "doc_lengths": doc_lens,
}
pyLDAvis._prepare.__num_dist_rows__ = __num_dist_rows__

In [6]:
LOG.info("Preparing data for pyLDAvis")
viz_data = pyLDAvis.prepare(**data, sort_topics=False)

In [7]:
viz_loc = 'vis250.html'
LOG.info(f"Writing visualization to {viz_loc}")
pyLDAvis.save_html(viz_data, str(viz_loc))

INFO:__main__:Writing visualization to vis250.html


In [10]:
import sys
sys.path.append('../src/')

In [11]:
from topic_modeling import get_bibcodes_with_embedding

In [8]:
embedding = tmp_a

In [13]:
infile = Path('../data/full_04_01_2020/kwds.jsonl')

In [15]:
map_loc = Path('../models/full_04_01_2020/mat_doc_mapping.csv')

In [16]:
mat_id_to_doc_id = pd.read_csv(map_loc, index_col=0)

/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
new_df = get_bibcodes_with_embedding(infile, embedding, mat_id_to_doc_id)

INFO:topic_modeling:Getting document bibcodes from ../data/full_04_01_2020/kwds.jsonl.
1161517it [00:29, 39569.34it/s]
INFO:topic_modeling:Reorder by descending topic scores where given topic is max
100%|██████████| 250/250 [01:36<00:00,  2.60it/s]


In [19]:
out_distrib = Path('topic_distribs_to_bibcodes.csv')

In [20]:
new_df.to_csv(out_distrib)

In [22]:
new_df.shape

(1159580, 252)

In [21]:
2 + 2

4

In [24]:
!du -sh topic_distribs_to_bibcodes.csv

3.7G	topic_distribs_to_bibcodes.csv
